## The network will also use NLP features obtained using Spark in the first part of the project.
This task is done by QuoraNLP_Step1_ClassicFeatures

In [22]:
import pickle
import pandas as pd
import os
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Embedding, LSTM, Add, Lambda, Multiply, BatchNormalization, GaussianNoise
from keras.layers import concatenate, add, multiply
from keras.utils import plot_model

#from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os

data_1 = pickle.load( open( './savedData/data_1.pkl', "rb" ) )
data_2 = pickle.load( open( './savedData/data_2.pkl', "rb" ) )
nb_words = pickle.load( open( './savedData/nb_words.pkl', "rb" ) )
embedding_matrix = pickle.load( open( './savedData/embedding_matrix.pkl', "rb" ) )
test_data_1 = pickle.load( open( './savedData/test_data_1.pkl', "rb" ) )
test_data_2 = pickle.load( open( './savedData/test_data_2.pkl', "rb" ) )

In [23]:
dataPath = "/user/wendygao16/fs_Quora/"
train_features = pd.read_csv(dataPath+'train_features.csv')
test_features  = pd.read_csv(dataPath+'test_features.csv')

In [24]:
print(data_1.shape, data_2.shape)
print(train_features.shape)
train_features = train_features.sort_values('id').reset_index(drop=True)
# make sure it ordery by id. For test set the id is testid.
train_features[1:10]

(404287, 30) (404287, 30)
(404288, 23)


,id,lWCount1,lWCount2,qWCount1,qWCount2,lLen1,lLen2,qLen1,qLen2,lWCount_ratio,...,qNgrams_1,qNgrams_2,qNgrams_3,lNgrams_1,lNgrams_2,lNgrams_3,qUnigram_ratio,lUnigram_ratio,tfidfDistance,is_duplicate
1,1,2,7,11,16,14,53,51,88,0.555556,...,4,2,1,1,0,0,0.044944,0.018519,615.275863,0
2,2,5,4,15,11,38,28,73,59,0.111111,...,4,1,0,3,0,0,0.054054,0.076923,202.907064,0
3,3,2,3,13,19,14,21,50,65,0.200000,...,0,0,0,0,0,0,0.000000,0.000000,423.533102,0
4,4,7,4,16,8,43,23,76,39,0.272727,...,4,0,0,0,0,0,0.051948,0.000000,1601.992142,0
5,5,6,5,20,21,30,35,86,90,0.090909,...,9,4,1,3,0,0,0.098901,0.083333,1256.299825,1
6,6,1,6,5,12,3,36,19,62,0.714286,...,0,0,0,0,0,0,0.000000,0.000000,485.216553,0
7,7,1,1,8,10,4,5,30,41,0.000000,...,4,1,0,0,0,0,0.095238,0.000000,91.627259,1
8,8,3,2,9,13,13,11,31,37,0.200000,...,6,4,2,2,0,0,0.157895,0.142857,0.000000,0
9,9,4,3,13,10,31,18,60,49,0.142857,...,4,1,0,2,0,0,0.065574,0.062500,221.256904,0


In [25]:
print(test_data_1.shape, test_data_2.shape)
print(test_features.shape)
test_features = test_features.sort_values('id').reset_index(drop=True)  # make sure it ordery by id. For test set the id is testid.
test_features.iloc[200:207]

(2345796, 30) (2345796, 30)
(2345796, 23)


,id,lWCount1,lWCount2,qWCount1,qWCount2,lLen1,lLen2,qLen1,qLen2,lWCount_ratio,...,qNgrams_1,qNgrams_2,qNgrams_3,lNgrams_1,lNgrams_2,lNgrams_3,qUnigram_ratio,lUnigram_ratio,tfidfDistance,is_duplicate
200,200,3,1,8,7,15,3,41,24,0.500000,...,3,1,0,0,0,0,0.071429,0.000000,269.072883,-1
201,201,3,2,10,11,16,10,39,43,0.200000,...,5,3,1,1,0,0,0.113636,0.058824,159.101342,-1
202,202,5,6,13,11,41,40,80,63,0.090909,...,1,0,0,0,0,0,0.012346,0.000000,977.433719,-1
203,203,3,4,11,11,19,24,43,48,0.142857,...,6,4,2,1,0,0,0.122449,0.040000,471.569633,-1
204,204,4,4,10,10,23,23,47,50,0.000000,...,4,1,0,3,2,1,0.078431,0.125000,202.626492,-1
205,205,7,3,17,13,55,19,109,85,0.400000,...,3,2,1,2,0,0,0.027273,0.035714,432.673679,-1
206,206,8,4,16,13,59,27,95,57,0.333333,...,3,1,0,0,0,0,0.031250,0.000000,1273.217695,-1


In [26]:
dataPath = "/user/wendygao16/fs_Quora/"
train = pd.read_csv(dataPath+'train_sample.csv', usecols=['question1', 'question2', 'id', 'is_duplicate'])
test = pd.read_csv(dataPath+'test_sample.csv', usecols=['test_id'])

train.dropna(inplace=True)
print(train.shape)
#train[404280:404290]
trainIds = train["id"].values
train_featuresIds = train_features["id"].values
diff = list(set(train_featuresIds) - set(trainIds))
diff

(404287, 4)


[363362]

In [27]:
# remove 363362
train_features.drop(train_features[train_features['id'] == 363362].index, inplace=True)

In [28]:
# drop 'is_duplicate', 'id'
test_features.drop(columns=['is_duplicate', 'id'], inplace=True)
test_features.iloc[200:207]

,lWCount1,lWCount2,qWCount1,qWCount2,lLen1,lLen2,qLen1,qLen2,lWCount_ratio,qWCount_ratio,...,qLen_ratio,qNgrams_1,qNgrams_2,qNgrams_3,lNgrams_1,lNgrams_2,lNgrams_3,qUnigram_ratio,lUnigram_ratio,tfidfDistance
200,3,1,8,7,15,3,41,24,0.500000,0.066667,...,0.261538,3,1,0,0,0,0,0.071429,0.000000,269.072883
201,3,2,10,11,16,10,39,43,0.200000,0.047619,...,0.048780,5,3,1,1,0,0,0.113636,0.058824,159.101342
202,5,6,13,11,41,40,80,63,0.090909,0.083333,...,0.118881,1,0,0,0,0,0,0.012346,0.000000,977.433719
203,3,4,11,11,19,24,43,48,0.142857,0.000000,...,0.054945,6,4,2,1,0,0,0.122449,0.040000,471.569633
204,4,4,10,10,23,23,47,50,0.000000,0.000000,...,0.030928,4,1,0,3,2,1,0.078431,0.125000,202.626492
205,7,3,17,13,55,19,109,85,0.400000,0.133333,...,0.123711,3,2,1,2,0,0,0.027273,0.035714,432.673679
206,8,4,16,13,59,27,95,57,0.333333,0.103448,...,0.250000,3,1,0,0,0,0,0.031250,0.000000,1273.217695


In [29]:
# get label
# drop 'is_duplicate', 'id'
y=train_features.is_duplicate
train_features.drop(columns=['is_duplicate', 'id'], inplace=True)
x_train = [data_1, data_2, train_features]
print(sum(y))
train_features.iloc[200:207]

149263


,lWCount1,lWCount2,qWCount1,qWCount2,lLen1,lLen2,qLen1,qLen2,lWCount_ratio,qWCount_ratio,...,qLen_ratio,qNgrams_1,qNgrams_2,qNgrams_3,lNgrams_1,lNgrams_2,lNgrams_3,qUnigram_ratio,lUnigram_ratio,tfidfDistance
200,3,3,13,10,15,15,51,45,0.000000,0.130435,...,0.062500,8,6,4,2,1,0,0.153846,0.125000,55.710405
201,9,13,18,33,57,79,98,150,0.181818,0.294118,...,0.209677,2,0,0,0,0,0,0.013245,0.000000,2518.464383
202,4,8,10,23,20,48,47,107,0.333333,0.393939,...,0.389610,6,4,3,3,2,1,0.055556,0.061224,326.193367
203,3,4,7,10,19,29,35,57,0.142857,0.176471,...,0.239130,3,1,0,2,0,0,0.051724,0.066667,280.842359
204,8,14,29,43,52,95,142,221,0.272727,0.194444,...,0.217631,6,0,0,2,0,0,0.027027,0.020833,1794.317502
205,5,5,23,18,31,29,95,74,0.000000,0.121951,...,0.124260,10,5,4,3,2,1,0.104167,0.093750,440.365833
206,2,2,9,6,9,11,34,25,0.000000,0.200000,...,0.152542,5,2,0,1,0,0,0.142857,0.083333,295.090072


## Create NN Graph using Keras

In [30]:
## Define embedding_layer 
from keras.layers import Embedding
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = data_1.shape[1]
#MAX_SEQUENCE_LENGTH = 30
BATCH_SIZE = 1024

embedding_layer = Embedding(nb_words, EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
(MAX_SEQUENCE_LENGTH, MAX_SEQUENCE_LENGTH, train_features.shape[1])

(30, 30, 21)

In [31]:
def get_model():
    embedding_layer = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)
    lstm_layer = LSTM(75, recurrent_dropout=0.2)

    sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = lstm_layer(embedded_sequences_1)

    sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1 = lstm_layer(embedded_sequences_2)

    features_input = Input(shape=(train_features.shape[1],), dtype="float32")
    features_dense = BatchNormalization()(features_input)
    features_dense = Dense(200, activation="relu")(features_dense)
    features_dense = Dropout(0.2)(features_dense)

    addition = add([x1, y1])
    minus_y1 = Lambda(lambda x: -x)(y1)
    merged = add([x1, minus_y1])
    merged = multiply([merged, merged])
    merged = concatenate([merged, addition])
    merged = Dropout(0.4)(merged)

    merged = concatenate([merged, features_dense])
    merged = BatchNormalization()(merged)
    merged = GaussianNoise(0.1)(merged)

    merged = Dense(150, activation="relu")(merged)
    merged = Dropout(0.2)(merged)
    merged = BatchNormalization()(merged)

    out = Dense(1, activation="sigmoid")(merged)

    model = Model(inputs=[sequence_1_input, sequence_2_input, features_input], outputs=out)
    model.compile(loss="binary_crossentropy", optimizer="nadam")
    return model
model = get_model()
#SVG(model_to_dot(nlpModel).create(prog='dot', format='svg'))

In [32]:
# ## Creating NN using Keras
# from keras.models import Sequential, Model
# from keras.layers import Dense, Input, Dropout, Embedding, LSTM, Add, Lambda, Multiply, BatchNormalization, GaussianNoise
# from keras.layers import concatenate
# from keras.utils import plot_model
# import pandas as pd
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot
# from keras.optimizers import SGD
# from keras.optimizers import Adam


# # left data branch
# input_1 = Input(shape = (MAX_SEQUENCE_LENGTH,), name='input_1')
# input_2 = Input(shape = (MAX_SEQUENCE_LENGTH,), name='input_2')

# embedding_1 = embedding_layer(input_1)
# embedding_2 = embedding_layer(input_2)

# lstm_1 = LSTM(30, recurrent_dropout = 0.1, name='lstm_1')(embedding_1)
# lstm_2 = LSTM(30, recurrent_dropout = 0.1, name='lstm_2')(embedding_2)

# add_1 = Add(name='add_1')([lstm_1,lstm_2])
# lambda_1 = Lambda(lambda x: -0.8*x, name='lambda_1')(lstm_1)
# add_2 = Add(name='add_2')([lambda_1, lstm_2])
# multiply_1 = Multiply(name='multiply_1')([add_2, add_2])

# concatenate_1 = concatenate([multiply_1, add_1], name='concatenate_1')
# dropout_2 = Dropout(rate=0.1, name='dropout_2')(concatenate_1)


# # right feature branch
# input_3 = Input(shape=(train_features.shape[1],), name='input_3')
# batchNormalization_1 = BatchNormalization(name='batchNormalization_1')(input_3)
# dense_1 = Dense(300, activation='relu', name='dense_1')(batchNormalization_1)
# dropout_1 = Dropout(rate=0.1, name='dropout_1')(dense_1)


# # join
# concatenate_2 = concatenate([dropout_2, dropout_1], name='concatenate_2')
# batchNormalization_2 = BatchNormalization(name='batchNormalization_2')(concatenate_2)
# gaussianNoise_1 = GaussianNoise(stddev=1, name='gaussianNoise_1')(batchNormalization_2)
# dense_2 = Dense(100, activation='relu', name='dense_2')(gaussianNoise_1)
# dropout_3 = Dropout(rate=0.1, name='dropout_3')(dense_2)

# # output
# batchNormalization_3 = BatchNormalization(name='batchNormalization_3')(dropout_3)
# output = Dense(1, activation='sigmoid',name='output')(batchNormalization_3)

# nlpModel = Model(inputs=[input_1, input_2, input_3], outputs=[output])
# plot_model(nlpModel, to_file='test_model1.png',show_shapes=True,show_layer_names=True)

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# nlpModel.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
# #nlpModel.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# SVG(model_to_dot(nlpModel).create(prog='dot', format='svg'))

## Fit

In [33]:
N_EPOCHS = 30
weights_path = 'weights'
if not os.path.exists(weights_path):
    os.makedirs(weights_path)
weights_path += '/unet_weights.hdf5'

if os.path.isfile(weights_path):
    model.load_weights(weights_path)
model_checkpoint = ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True)

In [2]:
earlystopping = EarlyStopping(monitor = "val_loss", patience = 5)
model.fit(x_train, y, batch_size=BATCH_SIZE, epochs=N_EPOCHS,
          verbose=1, shuffle=True, validation_split=0.01, callbacks=[model_checkpoint, earlystopping])

In [35]:
model.load_weights(weights_path)
pred = model.predict([test_data_1, test_data_2, test_features], verbose = 1)

2345796/2345796 [==============================] - 1426s 608us/step


In [36]:
prediction_prob = pred.ravel()
test_id = test.test_id
submission = pd.DataFrame({"test_id": test_id, "is_duplicate": prediction_prob})
submission.to_csv("submission1.csv", index=False)
print('Done')

Done
